In [5]:
# Load packages and set up default settings
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import datetime
import pickle

import data_load
import MT2

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [6]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [7]:
# Initialize all parameters
lambda_global = 10
lambda_penal = 1

M = 10
n_samples = 100

obj = 'MT2'
data_name = 'MNIST'

L_w = (lambda_global + lambda_penal) / M
L_beta = (1 + lambda_penal)

rho = 1e-2

n_communs = 1000
repo_step = 100
sync_step = 5

w0 = [torch.zeros(10, 784).to(device), torch.zeros(10).to(device)]

beta0 = []
for m in range(M):
    beta0.append([torch.zeros(10, 784).to(device), torch.zeros(10).to(device)])
    
w_list0 = []
for m in range(M):
    w_list0.append([torch.zeros(10, 784).to(device), torch.zeros(10).to(device)])

beta_list0 = []
for m in range(M):
    beta_list0.append([torch.zeros(10, 784).to(device), torch.zeros(10).to(device)])

In [8]:
print("L_w is: {}".format(L_w))
print("L_beta is {}".format(L_beta))
print("rho is {}".format(rho))
print("n_communs is {}".format(n_communs))
print("sync_step is {}".format(sync_step))

L_w is: 1.1
L_beta is 2
rho is 0.01
n_communs is 1000
sync_step is 5


In [5]:
train_loader_list, devices_train_list = data_load.data_prepare(data_name, n_devices=M, n_samples=n_samples)

In [7]:
prob1_list = [L_w / (L_w + L_beta), 0.7, 0.5, 0.3, 0.1]
MT2_MNIST_CDVR_iter_result = []
MT2_MNIST_CDVR_commun_result = []

for prob1 in prob1_list:
    print("prob1 is {:.5f}".format(prob1))
    
    eta = 1 / (8 * max(L_w/prob1, L_beta/(1-prob1)))
    
    print("eta is {:.5f}".format(eta))
    
    loss_MT2_MNIST_CDVR_iter, _, _ = MT2.train_CDVR_iter(w0, beta0, n_epochs, devices_train_list, 
                                                         train_loader_list, lambda_global, lambda_penal, 
                                                         repo_step, eta, prob1, rho, obj, data_name)
    MT2_MNIST_CDVR_iter_result.append(loss_MT2_MNIST_CDVR_iter)
    loss_MT2_MNIST_CDVR_commun, _, _ = MT2.train_CDVR_commun(w0, beta0, n_communs, devices_train_list, 
                                                             train_loader_list, lambda_global, lambda_penal, 
                                                             repo_step, eta, prob1, rho, obj, data_name)
    MT2_MNIST_CDVR_commun_result.append(loss_MT2_MNIST_CDVR_commun)

prob1 is 0.35484
eta is 0.04032
epoch: 1, loss: 25.3076999664, time pass: 0s | CDVR MT2 MNIST
epoch: 100, loss: 24.0192404747, time pass: 7s | CDVR MT2 MNIST
epoch: 200, loss: 23.2590580940, time pass: 13s | CDVR MT2 MNIST
epoch: 300, loss: 22.7626904488, time pass: 20s | CDVR MT2 MNIST
epoch: 400, loss: 22.2551648140, time pass: 27s | CDVR MT2 MNIST
epoch: 500, loss: 21.8298472404, time pass: 34s | CDVR MT2 MNIST
epoch: 600, loss: 21.4501642227, time pass: 40s | CDVR MT2 MNIST
epoch: 700, loss: 21.0857776642, time pass: 47s | CDVR MT2 MNIST
epoch: 800, loss: 20.7221816063, time pass: 54s | CDVR MT2 MNIST
epoch: 900, loss: 20.3782629967, time pass: 60s | CDVR MT2 MNIST
epoch: 1000, loss: 20.0436006546, time pass: 67s | CDVR MT2 MNIST
num_commun: 1, loss: 25.3076999664, time pass: 0s | CDVR MT2 MNIST
num_commun: 100, loss: 22.9563982010, time pass: 12s | CDVR MT2 MNIST
num_commun: 200, loss: 21.7258676529, time pass: 26s | CDVR MT2 MNIST
num_commun: 300, loss: 20.7431866646, time pass: 

In [8]:
with open("./result/MT2_MNIST_CDVR_iter_result.txt", "wb") as f:   #Pickling
    pickle.dump(MT2_MNIST_CDVR_iter_result, f)
    
with open("./result/MT2_MNIST_CDVR_commun_result.txt", "wb") as f:   #Pickling
    pickle.dump(MT2_MNIST_CDVR_commun_result, f)